In [9]:
#import modules
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import unicodedata

In [2]:
#sending get request
url = "https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{2}?Tid=6740"
arr = []

#creating for loop to get the data
for i in range(1, 101):
    x =f"https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{i}?Tid=6740"
    response = requests.get(url)
    soup = bs(response.content, "lxml")  
    div = soup.findAll("div", class_="item-cell")
    for i in div:
        arr.append(i)

In [5]:
#creating functions to clean data

# item-features
def extractList(g):
    k= {}
    for i in g:
        y = i.text.rsplit(":")
        if len(y)>1:
            k[y[0]]=y[1]
    return k

#price
def priceCurrent(k):
    l =unicodedata.normalize("NFKD",k.rsplit("$")[-1])
    return l.rsplit(" ")[0]

#check for nonetype
def isNone(g):
    if g is None:
        return "None"
    else:
        return g.text

In [6]:
#apped into data into a new array
arr_data  = []
for i in arr:
    flags_body = i.find("div", class_="flags flags-right flags-red is-invert")
    item_title = i.find("a", class_="item-title")
    item_promo = i.find("p", class_="item-promo")
    price_ship = i.find("li", class_="price-ship")
    price_offers = i.find("a", class_="price-current-num")
    
    hello = {
    "flags-body": isNone(flags_body),
    "item-title": isNone(item_title),
    "item-promo": isNone(item_promo),
    "item-features": extractList(i.find("ul", class_="item-features")),
    "item-price": priceCurrent(i.find("li", class_="price-current").text),
    "price-ship": isNone(price_ship),
    "price-offers": isNone(price_offers),
    }
    arr_data.append(hello)

In [11]:
#convert array to data
df = pd.DataFrame(arr_data)
df.head()

,flags-body,item-title,item-promo,item-features,item-price,price-ship,price-offers
0,Lowest price,Acer Laptop Aspire 5 Intel Core i7 12th Gen 12...,"Eligible for Black Friday Price Protection, Se...","{'Type': ' Home / Personal', 'Resolution': ' 1...",599.99,Free Shipping,(2 Offers)
1,None,MSI Laptop Intel Core i7 12th Gen 1280P (1.80G...,None,"{'Type': ' Work / Business', 'Resolution': ' 1...",649.00,Free Shipping,(2 Offers)
2,None,"Lenovo IdeaPad Slim 7 Carbon 14"" 2.8K OLED Tou...","Eligible for Black Friday Price Protection, Se...","{'Type': ' Home / Personal', 'Resolution': ' 2...",799.99,Free Shipping,None
3,None,"ASUS VivoBook 16X OLED Laptop, 16"" 4K OLED 16:...","Eligible for Black Friday Price Protection, Se...",{'Type': ' Content Creation / Home / Personal'...,849.99,Free Shipping,None
4,None,Acer Laptop Aspire 5 Intel Core i5 11th Gen 11...,None,"{'Type': ' Home / Personal', 'Resolution': ' 1...",499.99,Free Shipping,(4 Offers)


In [13]:
#convert to csv file 
from pathlib import Path

filepath = Path("./data.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath)
